<a href="https://colab.research.google.com/github/nonyeezeh/Research-Project-Code/blob/main/NN_Sparse_1_10_Relu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
pip install pgmpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 21.4 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
from pgmpy.models import BayesianNetwork
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD
from pgmpy.sampling import BayesianModelSampling
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras import layers, models
from pgmpy.estimators import HillClimbSearch, BicScore, AICScore, MaximumLikelihoodEstimator
from pgmpy.inference import VariableElimination
from sklearn.metrics import accuracy_score
from scipy.stats import entropy
import os
import matplotlib.pyplot as plt

# Bayesian Network Data Generation 1000, 2000, ..., 10000 Samples (sparse)

In [ ]:
# Define the mappings for IR, EI, SP
ir_map = {0: 'low', 1: 'medium', 2: 'high'}
ei_map = {0: 'poor', 1: 'average', 2: 'good'}
sp_map = {0: 'decrease', 1: 'stable', 2: 'increase'}

# Define the sparse Bayesian Network
sparse_model = BayesianNetwork([('IR', 'SP'), ('EI', 'SP')])

# Function to generate CPDs for the sparse model
def generate_cpds_sparse():
    # Generate probabilities for IR (unconditional)
    ir_probs = np.random.rand(3)
    ir_probs /= ir_probs.sum()  # Normalize to make it a valid probability distribution

    # Generate unconditional probabilities for EI (no dependency on IR)
    ei_probs = np.random.rand(3)
    ei_probs /= ei_probs.sum()

    # Generate conditional probabilities for SP given IR and EI
    sp_probs = np.random.rand(3, 3, 3)
    sp_probs /= sp_probs.sum(axis=0, keepdims=True)

    sp_probs_reshaped = sp_probs.reshape(3, -1)

    return ir_probs, ei_probs, sp_probs_reshaped

# Save probabilities in a single CSV file
def save_probabilities_sparse(ir_probs, ei_probs, sp_probs, filename):
    # Create a DataFrame for IR probabilities
    ir_df = pd.DataFrame({
        'IR_State': ['low', 'medium', 'high'],
        'IR_Prob': ir_probs
    })

    # Create a DataFrame for EI probabilities (since it's not conditional on IR)
    ei_df = pd.DataFrame({
        'EI_State': ['poor', 'average', 'good'],
        'EI_Prob': ei_probs
    })

    # Create a DataFrame for SP given IR and EI probabilities
    sp_df = pd.DataFrame(sp_probs, columns=[
        'SP_given_IR_low_EI_poor', 'SP_given_IR_low_EI_average', 'SP_given_IR_low_EI_good',
        'SP_given_IR_medium_EI_poor', 'SP_given_IR_medium_EI_average', 'SP_given_IR_medium_EI_good',
        'SP_given_IR_high_EI_poor', 'SP_given_IR_high_EI_average', 'SP_given_IR_high_EI_good'
    ])
    sp_df['SP_State'] = ['decrease', 'stable', 'increase']

    # Combine all data into a single DataFrame
    combined_df = pd.concat([ir_df, ei_df, sp_df], axis=1)

    # Save the combined DataFrame as a single CSV file
    combined_df.to_csv(filename, index=False)

# Save outcomes in a CSV file
def save_outcomes_sparse(data_sparse, filename):
    data_sparse['IR'] = data_sparse['IR'].map(ir_map)
    data_sparse['EI'] = data_sparse['EI'].map(ei_map)
    data_sparse['SP'] = data_sparse['SP'].map(sp_map)
    data_sparse.to_csv(filename, index=False)

# Generate datasets for different sample sizes for the sparse model
sample_sizes = range(1000, 10500, 1000)
for size in sample_sizes:
    # Generate the CPDs
    ir_probs, ei_probs, sp_probs_reshaped = generate_cpds_sparse()

    # Define CPDs for the sparse model
    cpd_ir = TabularCPD(variable='IR', variable_card=3, values=[[ir_probs[0]], [ir_probs[1]], [ir_probs[2]]])
    cpd_ei_sparse = TabularCPD(variable='EI', variable_card=3, values=[[ei_probs[0]], [ei_probs[1]], [ei_probs[2]]])
    cpd_sp_sparse = TabularCPD(variable='SP', variable_card=3,
                               values=sp_probs_reshaped,
                               evidence=['IR', 'EI'], evidence_card=[3, 3])

    sparse_model.add_cpds(cpd_ir, cpd_ei_sparse, cpd_sp_sparse)

    # Check if the model is valid
    assert sparse_model.check_model()

    # Generate samples
    sampler_sparse = BayesianModelSampling(sparse_model)
    data_sparse = sampler_sparse.forward_sample(size=size)

    # Save probabilities in one file
    save_probabilities_sparse(ir_probs, ei_probs, sp_probs_reshaped, f'probabilities_sparse_{size}.csv')

    # Save outcomes (low, medium, high) in another file
    save_outcomes_sparse(data_sparse, f'outcomes_sparse_{size}.csv')

# Notify the user that the process is done
print("Data generation and saving complete for the sparse model!")

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Data generation and saving complete for the sparse model!


# Hypothesis Model 500, 1000, 1500, ..., 10000 Samples (sparse) 1 hidden Layer, 10 Neurons Relu

In [ ]:
# Sample sizes to loop through
sample_sizes = range(1000, 10500, 1000)

# Define the Neural Network architecture
def create_nn_model(hidden_layers=1, nodes_per_layer=10):
    model = models.Sequential()

    # Input layer (2 input features: IR_encoded and EI_encoded)
    model.add(layers.InputLayer(input_shape=(2,)))

    # Hidden layers
    for layer_num in range(hidden_layers):
        model.add(layers.Dense(nodes_per_layer, activation='relu', name=f"hidden_layer_{layer_num + 1}"))

    # Output layer (3 classes: decrease, stable, increase)
    model.add(layers.Dense(3, activation='softmax', name="output_layer"))

    # Compile the model
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

# Loop through each sample size
for size in sample_sizes:
    # Load data for the current sample size (adjust the file paths for sparse data)
    outcomes_file = f'outcomes_sparse_{size}.csv'

    df = pd.read_csv(outcomes_file)

    # Manually encode categorical variables for IR, EI, and SP
    ir_map = {'low': 0, 'medium': 1, 'high': 2}
    ei_map = {'poor': 0, 'average': 1, 'good': 2}
    sp_map = {'decrease': 0, 'stable': 1, 'increase': 2}

    df['IR_encoded'] = df['IR'].map(ir_map)
    df['EI_encoded'] = df['EI'].map(ei_map)
    df['SP_encoded'] = df['SP'].map(sp_map)

    # Features (IR and EI) and labels (SP)
    X = df[['IR_encoded', 'EI_encoded']]
    y = df['SP_encoded']

    # Refresh the data split for each iteration
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, shuffle=False, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, shuffle=False, random_state=42)

    # Show split confirmation
    print(f"\nSample size: {size}")
    print("Training Data:", X_train.shape, y_train.shape)
    print("Validation Data:", X_val.shape, y_val.shape)
    print("Test Data:", X_test.shape, y_test.shape)

    # Create the Neural Network model
    nn_model = create_nn_model(hidden_layers=1, nodes_per_layer=10)

    # Train the model
    history = nn_model.fit(X_train, y_train,
                           epochs=50,
                           batch_size=32,
                           validation_data=(X_val, y_val),
                           verbose=0)  # Set verbose=0 to avoid too much output

    # Evaluate on the validation set
    val_loss, val_accuracy = nn_model.evaluate(X_val, y_val, verbose=0)
    print(f"Validation Accuracy for {size} samples: {val_accuracy:.4f}")

    # Evaluate on the test set
    test_loss, test_accuracy = nn_model.evaluate(X_test, y_test, verbose=0)
    print(f"Test Accuracy for {size} samples: {test_accuracy:.4f}")

    # Make predictions on the test set
    predictions = nn_model.predict(X_test)

    # Convert the predicted probabilities to class labels
    predicted_classes = predictions.argmax(axis=1)

    # Create a list to map integers back to the original SP labels
    sp_reverse_map = ['decrease', 'stable', 'increase']

    # Convert the predicted classes to the original labels
    predicted_labels = [sp_reverse_map[label] for label in predicted_classes]

    # Create a DataFrame for the predicted probabilities
    probs_df = pd.DataFrame(predictions, columns=['Prob_decrease', 'Prob_stable', 'Prob_increase'])

    # Output the IR, EI, predicted SP, and the NN probabilities
    result_df = pd.DataFrame({
        'IR': df['IR'][:len(predicted_labels)],  # IR column from the original dataframe
        'EI': df['EI'][:len(predicted_labels)],  # EI column from the original dataframe
        'Predicted_SP': predicted_labels         # Predicted SP labels
    })

    # Combine the result with the predicted probabilities
    combined_df = pd.concat([result_df, probs_df.reset_index(drop=True)], axis=1)

    # Show the first few rows of the results for this sample size
    print(f"\nPredicted Results and Probabilities for {size} samples (First 5 rows):")
    print(combined_df.head(15))

# After the loop is done, print this message
print("\nLooping through all sparse sample sizes complete!")

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(



Sample size: 500
Training Data: (350, 2) (350,)
Validation Data: (75, 2) (75,)
Test Data: (75, 2) (75,)
Validation Accuracy for 500 samples: 0.4133
Test Accuracy for 500 samples: 0.4933
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step

Predicted Results and Probabilities for 500 samples (First 5 rows):
        IR       EI Predicted_SP  Prob_decrease  Prob_stable  Prob_increase
0   medium     good     decrease       0.496978     0.344048       0.158974
1   medium     good     decrease       0.457540     0.423755       0.118705
2     high     good     decrease       0.496978     0.344048       0.158974
3      low     good     decrease       0.496978     0.344048       0.158974
4     high     good     decrease       0.496978     0.344048       0.158974
5     high     good     decrease       0.496978     0.344048       0.158974
6      low     good     increase       0.259504     0.292808       0.447688
7   medium     good     increase       0.259504     0.292808       0.447688
8     high     good   

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Validation Accuracy for 1000 samples: 0.5467
Test Accuracy for 1000 samples: 0.5400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 

Predicted Results and Probabilities for 1000 samples (First 5 rows):
        IR       EI Predicted_SP  Prob_decrease  Prob_stable  Prob_increase
0   medium  average       stable       0.168973     0.620075       0.210952
1     high  average       stable       0.168973     0.620075       0.210952
2     high     poor       stable       0.168973     0.620075       0.210952
3      low  average       stable       0.168973     0.620075       0.210952
4     high  average     increase       0.109114     0.185783       0.705103
5      low  average       stable       0.185938     0.466974       0.347088
6     high     good       stable       0.168973     0.620075       0.210952
7     high  average       stable       0.168973     0.620075       0.210952
8      low  average     increase       0.123192     0.184484       0.692323
9      low  average     increase       0.123192 

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


...

Validation Accuracy for 9000 samples: 0.4867
Test Accuracy for 9000 samples: 0.5052
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step

Predicted Results and Probabilities for 9000 samples (First 5 rows):
        IR       EI Predicted_SP  Prob_decrease  Prob_stable  Prob_increase
0      low     poor     increase       0.085371     0.399740       0.514890
1   medium     poor     increase       0.085371     0.399740       0.514890
2     high  average     decrease       0.411834     0.237105       0.351061
3      low  average       stable       0.201882     0.463360       0.334758
4      low  average     decrease       0.482289     0.388086       0.129625
5      low  average     decrease       0.483693     0.394013       0.122293
6     high  average     increase       0.294175     0.280271       0.425554
7      low  average       stable       0.011546     0.658772       0.329682
8      low     poor       stable       0.201882     0.463360       0.334758
9      low  average       stable       0.201882

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Validation Accuracy for 9500 samples: 0.4632
Test Accuracy for 9500 samples: 0.4498
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

Predicted Results and Probabilities for 9500 samples (First 5 rows):
        IR       EI Predicted_SP  Prob_decrease  Prob_stable  Prob_increase
0   medium  average     increase       0.381421     0.194572       0.424007
1     high     poor     increase       0.371281     0.255671       0.373048
2     high     poor     increase       0.381421     0.194572       0.424007
3      low  average     increase       0.178109     0.328714       0.493177
4   medium     poor     decrease       0.427614     0.216051       0.356335
5   medium     good       stable       0.310035     0.616957       0.073009
6     high     poor       stable       0.310035     0.616957       0.073009
7      low     poor       stable       0.310035     0.616957       0.073009
8      low  average     increase       0.371281     0.255671       0.373048
9      low     poor     decrease       0.427614

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Validation Accuracy for 10000 samples: 0.4580
Test Accuracy for 10000 samples: 0.4500
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

Predicted Results and Probabilities for 10000 samples (First 5 rows):
        IR       EI Predicted_SP  Prob_decrease  Prob_stable  Prob_increase
0   medium     poor     decrease       0.398184     0.326436       0.275380
1      low     poor     decrease       0.398184     0.326436       0.275380
2     high     poor     increase       0.298641     0.189217       0.512142
3     high  average     increase       0.298641     0.189217       0.512142
4      low  average     decrease       0.539115     0.299848       0.161037
5     high     poor     increase       0.325414     0.275082       0.399504
6      low     good     decrease       0.398184     0.326436       0.275380
7   medium     poor       stable       0.336777     0.370899       0.292323
8     high  average     decrease       0.398184     0.326436       0.275380
9   medium  average     decrease       0.539

# K-L Divergence NN Sparse Data

In [ ]:
# Define the function to save K-L divergence and std dev to a file
def save_kl_divergence(sample_size, kl_div_value, std_kl_div_value, first_run=False):
    file_name = 'kl_div_NN_1_10_sparse.csv'

    # If it's the first run, write the headers
    if first_run:
        with open(file_name, 'w') as f:
            f.write('Size,NN_Sparse_1_10_Entropy,Std_Dev\n')  # Write headers for the CSV

    # Append the K-L divergence and std dev for this sample size to the file
    with open(file_name, 'a') as f:
        f.write(f"{sample_size},{kl_div_value:.4f},{std_kl_div_value:.4f}\n")

# Sample sizes to loop through
sample_sizes = range(1000, 10500, 1000)  # Loop from 500 to 10,000 in increments of 500

# Loop through each sample size
first_run = True  # Flag to indicate the first run for writing headers

for sample_size in sample_sizes:
    # Load the ground truth probabilities for the current sample size
    ground_truth_probs_file = f'probabilities_sparse_{sample_size}.csv'
    df_gt_probs = pd.read_csv(ground_truth_probs_file)

    # Placeholder to store K-L divergence values
    kl_divergences = []

    # Load the predictions from memory (already generated by the NN)
    outcomes_file = f'outcomes_sparse_{sample_size}.csv'
    df = pd.read_csv(outcomes_file)

    # Manually encode categorical variables for IR and EI
    ir_map = {'low': 0, 'medium': 1, 'high': 2}
    ei_map = {'poor': 0, 'average': 1, 'good': 2}

    df['IR_encoded'] = df['IR'].map(ir_map)
    df['EI_encoded'] = df['EI'].map(ei_map)

    # Assuming predictions were saved in memory from the NN run
    predictions = nn_model.predict(df[['IR_encoded', 'EI_encoded']])

    # Loop through the test set predictions
    for i in range(len(df)):
        # Neural Network predicted probabilities for SP (decrease, stable, increase)
        predicted_probs = predictions[i]

        # Get the IR and EI values for the current sample
        ir_value = df.iloc[i]['IR_encoded']
        ei_value = df.iloc[i]['EI_encoded']

        # Map encoded values back to original labels
        ir_value = {0: 'low', 1: 'medium', 2: 'high'}[ir_value]
        ei_value = {0: 'poor', 1: 'average', 2: 'good'}[ei_value]

        # Get the corresponding ground truth probabilities for SP given IR and EI
        col_prefix = f'SP_given_IR_{ir_value}_EI_{ei_value}'
        ground_truth_probs = df_gt_probs.filter(like=col_prefix).values.flatten()

        # Ensure the probabilities are non-zero to avoid division by zero
        epsilon = 1e-10
        ground_truth_probs = np.clip(ground_truth_probs, epsilon, 1)

        # Compute K-L divergence (Neural Network vs Ground Truth)
        kl_div = entropy(predicted_probs, ground_truth_probs)
        kl_divergences.append(kl_div)

    # Calculate the average K-L divergence over all samples for the current sample size
    average_kl_divergence = np.mean(kl_divergences)

    # Calculate the standard deviation of the K-L divergence values
    std_kl_divergence = np.std(kl_divergences)

    # Save the K-L divergence and std dev value to a CSV file
    save_kl_divergence(sample_size, average_kl_divergence, std_kl_divergence, first_run=first_run)

    # Print confirmation
    print(f"Average K-L Divergence for {sample_size} samples: {average_kl_divergence:.4f}, Std Dev: {std_kl_divergence:.4f}")

    # After the first run, set `first_run` to False
    first_run = False

# Once all sample sizes are processed, the K-L divergences and std devs will be saved in 'kl_div_NN_1_10_sparse.csv'
print("\nK-L divergence and standard deviation calculations complete.")

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Average K-L Divergence for 500 samples: 0.2801, Std Dev: 0.2647
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Average K-L Divergence for 1000 samples: 0.4282, Std Dev: 0.3127
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Average K-L Divergence for 1500 samples: 0.3050, Std Dev: 0.3148
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Average K-L Divergence for 2000 samples: 0.2219, Std Dev: 0.1781
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Average K-L Divergence for 2500 samples: 0.0796, Std Dev: 0.0306
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Average K-L Divergence for 3000 samples: 0.2201, Std Dev: 0.2397
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Average K-L Divergence for 3500 samples: 0.6165, Std Dev: 0.5699
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Average K-L Divergence for 4000 samples: 0.2679, Std Dev: 0.2148
141/141 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Average K-L Divergence for 4500 samples: 0.1806, Std Dev: 0.1895
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Average K-L Dive